# 과제: 네이버 영화 정보 및 평점 크롤링

- 대상: 예매순 상위 5개의 현재 상영 중인 영화
- 수집할 항목: 영화 제목, 주연배우 3인, 네티즌 평점, 관람객 평점, 기자/평론가 평점, 관람객 별점 리뷰 20건 공감순으로(평점, 작성자닉네임, 리뷰본문)

In [11]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [39]:
def return_soup(url):
    html = requests.get(url).text
    soup = BeautifulSoup(html, from_encoding='utf-8')
    return soup

In [40]:
def make_movie_code():
    # 영화 코드 정보를 불러오는 함수 작성
    movie_code = []
    soup = return_soup('https://movie.naver.com/movie/running/current.nhn')
    for title in soup.find_all('dt', attrs={'class':'tit'}):
        movie_code.append(title.find('a').get('href').split("?")[1])
    return movie_code

### 1. 예매순 상위 5개의 현재 상영 중인 영화 가져오기

영화 제목, 주연배우 3인

In [14]:
def movie_title_url_actor():
    
    soup = return_soup('https://movie.naver.com/movie/running/current.nhn')
    movie_dict = {"title":[], "actors":[]} # 나중에 dataframe으로도 쉽게 바꿀 수 있도록 딕셔너리에 저장

    # 영화 제목에 대한 데이터 크롤링
    # class가 tit인 dt tag 정보를 가져온다.
    for title in soup.find_all('dt', {"class":"tit"}):
        movie_dict["title"].append(title.find("a").text.strip())

    # 영화 출연인에 대한 데이터 크롤링 (영화단위로 묶임)
    for movie in soup.find_all('dl', {"class":"info_txt1"}):
        category_len = len(movie.find_all('dt')) # 세부목차의 개수 
        ## (개요, 감독)으로 구성된 영화 -> 출연인을 np.nan으로 대체
        if category_len == 2:
            movie_dict["actors"].append(np.nan)
        ## (개요, 감독, 출연)으로 구성된 영화 -> 출연인 정보 반영
        elif category_len == 3:
            actors = movie.find_all("span", {"class":"link_txt"})[2].find_all("a")
            actors_list = []
            for actor in actors:
                actors_list.append(actor.text)
            movie_dict["actors"].append(actors_list)
            
    return movie_dict

### 2. 해당 영화의 평점 가져오기

네티즌 평점, 관람객 평점, 기자/평론가 평점

In [15]:
def get_grade():
    
    score_dict = {'viewer_score':[], 'spc_score':[], "netizen_score":[]} # 결과를 dataframe으로 쉽게 변환할 수 있도록 딕셔너리에 저장
    movie_code = make_movie_code()
    soup = return_soup('https://movie.naver.com/movie/running/current.nhn')
    
    # 굳이 네티즌평점을 분리해서 구한 이유는
    # 관람객 평점, 기자/평론가 평점은 없는 경우가 있지만 네티즌평점은 모든 경우에 있어 데이터가 다 존재하기 때문에! 
    star_list = soup.find_all(name='div', attrs={"class":"star_t1"})
    for i, star in enumerate(star_list):
        # 네티즌 평점과 예매율 분리
        try:
            star.find(name='span', attrs={'class':'txt'}).text
            continue;
        except:
            score_dict['netizen_score'].append(float(star.find(name='span', attrs={"class":"num"}).text))    
            
            
    # 링크 하나하나 방문해가면서 평점 데이터 얻기
    for code in movie_code:
        movie = return_soup("http://movie.naver.com/movie/bi/mi/basic.nhn?" + code)
        scores = movie.find_all('div', attrs={'class':'star_score'})
        score_list = []
        
        # viewer score
        if (scores[0].text.strip() != '관람객 평점 없음'):
            score_list.append(float("".join([i.text for i in scores[0].find_all("em")])))
        else:
            score_list.append(np.nan)
                
        # spc score
        # 다음 scores[2] 역시 빈 리스트가 아니여야 spc_score로서의 의미를 갖는다.
        if (scores[1].find_all("em") and scores[2].find_all("em")):
            score_list.append(float("".join([i.text for i in scores[1].find_all("em")])))
        else:
            score_list.append(np.nan)

        viewer_score, spc_score = score_list
        score_dict["viewer_score"].append(viewer_score)
        score_dict["spc_score"].append(spc_score)

    return score_dict

### 3. 관람객 평점 공감순 20건 가져오기

평점, 평점 작성자 닉네임, 리뷰 본문

In [28]:
def get_review():
    
    movie_code = make_movie_code()
    review_dict = {"score":[], "writer":[], "content":[]}
    
    for code in movie_code:
        soup = return_soup(f"https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?{code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false")
        score_list = []; writer_list = []; content_list = []
        
        # 평점을 추출하는 코드
        scores = soup.find_all("div", attrs={"class":"star_score"})
        for score in scores:
            score_list.append(int(score.find("em").text)) 
        
        # 평점 작성자와 리뷰 본문을 추출하는 코드
        reviews = soup.find_all("div", attrs={"class":"score_reple"})
        for review in reviews:
            span_len = len(review.find_all("span"))
            # 종종 길게 작성된 리뷰의 경우 "더보기"를 눌러야 전체 리뷰내용을 확인할 수 있는 경우가 있다.
            # 따라서 길이가 긴 경우는 try에서 리뷰내용을 크롤링해오고,
            # 길이가 짧아서 바로 리뷰 내용을 볼 수 경우는 except로 가서 크롤링해오도록 코드 작성하였다.
            try:
                content_list.append(review.find_all("span")[span_len-2].find("a").get("data-src").strip())
            except:
                content_list.append(review.find_all("span")[span_len-2].text.strip())
            writer_list.append(review.find_all("span")[span_len-1].text.strip())
        
        # 하나의 영화에 대한 리뷰를 한 곳에 모으기 위해 score_list, writer_list, content_list의 리스트를 만들어 리뷰를 담은 후,
        # 이 리스트를 딕셔너리의 value에 append 하는 식으로 데이터를 저장하였다.
        review_dict["score"].append(score_list)
        review_dict["writer"].append(writer_list)
        review_dict["content"].append(content_list)
        
    return review_dict

In [17]:
review_dict = get_reviews()

### 4. 저장하기

In [35]:
def save(title_dict, grade_dict, review_dict):
    
    # 데이터프레임으로 만들어서 병합하고, 그 객체를 반환
    title = pd.DataFrame(title_dict)
    grade = pd.DataFrame(grade_dict)
    review = pd.DataFrame(review_dict)
    
    df = pd.concat([title, grade, review], axis=1)
    return df

### 5. 크롤링하기

In [22]:
title = movie_title_url_actor()

In [23]:
pd.DataFrame(title).head()

,title,actors
0,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우, 윤여정, 정만식, 진경, 신현빈, 정가람]"
1,정직한 후보,"[라미란, 김무열, 나문희, 윤경호]"
2,1917,"[조지 맥케이, 딘-찰스 채프먼]"
3,작은 아씨들,"[시얼샤 로넌, 엠마 왓슨, 플로렌스 퓨, 엘리자 스캔런, 티모시 샬라메]"
4,클로젯,"[하정우, 김남길, 허율, 김시아]"


In [25]:
print(pd.DataFrame(title).shape)

(86, 2)


In [27]:
grade = get_grade()

In [30]:
pd.DataFrame(grade).head()

,viewer_score,spc_score,netizen_score
0,9.01,6.71,7.63
1,8.74,5.38,7.76
2,9.52,7.67,9.11
3,9.26,8.00,8.91
4,8.47,5.50,6.88


In [32]:
print(pd.DataFrame(grade).shape)

(86, 3)


In [29]:
review = get_review()

In [33]:
pd.DataFrame(review).head()

,score,writer,content
0,"[10, 10, 10, 9, 9, 10, 9, 10, 9, 10]","[bohemian(mabu****), 최정규(cjg4****), 달다(fxko***...","[난 전도연의 화류계 캐릭터가 좋다. 무뢰한, 너는 내 운명, 카운트다운...그리고..."
1,"[6, 10, 10, 10, 5, 2, 9, 10, 9, 10]","[lililli(wken****), 김다솜(kmjd****), 하니(what****...","[솔직히 그렇게 엄청 웃긴지는 모르겠어요, 너무 재밌었어용ㅠ 라미란 짱멋…, 엥 댓..."
2,"[8, 10, 10, 10, 10, 10, 10, 10, 10, 10]","[hose(jsd9****), 6월의매(hawk****), 소원열차(alst****...","[충무로: 이거 어케하는거냐?, 이 영화는 미쳤다. 넷플릭스가 일상화된 시대에 극장..."
3,"[10, 10, 8, 10, 10, 10, 9, 10, 8, 10]","[문덕현(mdh0****), 이쁘구나(ehsk****), 10(loli****), ...","[재밌다고 느끼면 추천을 아니다 비추천, 왜 여성은 사랑을 해야하냐며 말하면서도 사..."
4,"[10, 10, 1, 10, 10, 9, 2, 10, 9, 10]","[깜땀(qw13****), gilbest(sjca****), KCJ0212(lahe...","[김남길 미치게 섹시하다 진심, 김남길 배우 연기천재 인증. 몰입력 흡입력 ㅎㄷㄷ함..."


In [34]:
print(pd.DataFrame(review).shape)

(86, 3)


In [37]:
df = save(title, grade, review)

In [38]:
df

,title,actors,viewer_score,spc_score,netizen_score,score,writer,content
0,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우, 윤여정, 정만식, 진경, 신현빈, 정가람]",9.01,6.71,7.63,"[10, 10, 10, 9, 9, 10, 9, 10, 9, 10]","[bohemian(mabu****), 최정규(cjg4****), 달다(fxko***...","[난 전도연의 화류계 캐릭터가 좋다. 무뢰한, 너는 내 운명, 카운트다운...그리고..."
1,정직한 후보,"[라미란, 김무열, 나문희, 윤경호]",8.74,5.38,7.76,"[6, 10, 10, 10, 5, 2, 9, 10, 9, 10]","[lililli(wken****), 김다솜(kmjd****), 하니(what****...","[솔직히 그렇게 엄청 웃긴지는 모르겠어요, 너무 재밌었어용ㅠ 라미란 짱멋…, 엥 댓..."
2,1917,"[조지 맥케이, 딘-찰스 채프먼]",9.52,7.67,9.11,"[8, 10, 10, 10, 10, 10, 10, 10, 10, 10]","[hose(jsd9****), 6월의매(hawk****), 소원열차(alst****...","[충무로: 이거 어케하는거냐?, 이 영화는 미쳤다. 넷플릭스가 일상화된 시대에 극장..."
3,작은 아씨들,"[시얼샤 로넌, 엠마 왓슨, 플로렌스 퓨, 엘리자 스캔런, 티모시 샬라메]",9.26,8.00,8.91,"[10, 10, 8, 10, 10, 10, 9, 10, 8, 10]","[문덕현(mdh0****), 이쁘구나(ehsk****), 10(loli****), ...","[재밌다고 느끼면 추천을 아니다 비추천, 왜 여성은 사랑을 해야하냐며 말하면서도 사..."
4,클로젯,"[하정우, 김남길, 허율, 김시아]",8.47,5.50,6.88,"[10, 10, 1, 10, 10, 9, 2, 10, 9, 10]","[깜땀(qw13****), gilbest(sjca****), KCJ0212(lahe...","[김남길 미치게 섹시하다 진심, 김남길 배우 연기천재 인증. 몰입력 흡입력 ㅎㄷㄷ함..."
5,기생충,"[송강호, 이선균, 조여정, 최우식, 박소담, 이정은, 장혜진]",9.07,9.06,8.49,"[10, 10, 10, 10, 10, 9, 8, 10, 10, 8]","[brilliant_AKA_밝음(bril****), 김희정(priv****), 리오...","[비에 젖지 않는 고급 장난감 텐트와, 비에 젖다 못해 잠겨버리는 반지하 가구, 최..."
6,수퍼 소닉,"[짐 캐리, 제임스 마스던, 벤 슈와츠, 티카 섬터]",8.69,5.00,8.43,"[9, 8, 8, 10, 10, 10, 10, 8, 8, 10]","[STARK(lshc****), 리링(skyo****), 스테고CH(lcm2****...",[소닉 성형이 신의 한수... 제작진이 성형시키기 전엔 진짜 말그대로 파란색 고슴도...
7,"하이, 젝시","[아담 드바인, 로즈 번]",9.33,5.50,6.88,"[6, 8, 8, 10, 10, 8, 6, 10, 9, 8]","[napa****, fsf2fff(dnjs****), 사랑사랑사랑사랑(kckz***...",[그냥저냥 킬링타임용. 이럴거면 아예 청불 등급에 더 쎈 수위로 나왔으면 좋았을듯....
8,숀더쉽 더 무비: 꼬마 외계인 룰라!,"[저스틴 플레쳐, 아멜리아 비테일]",10.00,6.75,9.49,"[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]","[JINs(sjho****), xx(s_he****), zinzin(high****...","[자녀들과 같이 보기에 아주 건전하고 재미있는 영화임! 강추함!!, 미마 쥼쥼 ㅠㅠ..."
9,조조 래빗,"[스칼렛 요한슨, 로만 그리핀 데이비스, 타이카 와이티티, 토마신 맥켄지]",9.08,7.17,9.26,"[10, 10, 10, 10, 10, 9, 8, 10, 8, 10]","[2023032(yjpa****), 레이첼(twee****), 짬뽕지존(ksks**...","[연기, 연출, 스토리, 영상미 뭐 하나 부족한게 없다 진짜 최고, 영화의 웃음 뒤..."
